In [1]:
from src.indexing import build_index_from_url, build_chunks_from_tree, EmbeddingManager

[nltk_data] Downloading package punkt to ../../data/ntlk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to ../../data/ntlk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
tree, folder_path = await build_index_from_url("https://en.wikipedia.org/wiki/peru", 1)
chunks = build_chunks_from_tree(tree)

Created a chunk of size 607, which is longer than the specified 512
Created a chunk of size 660, which is longer than the specified 512


In [3]:
em = EmbeddingManager(index_dir = folder_path)
# Generate embeddings
index = em(chunks, "2website_embbed_index")

Loaded index with 337 vectors
